# Keyboard Modes

> Configuration for keyboard modes that change navigation and action behavior.

In [ ]:
#| default_exp core.modes

In [ ]:
#| export
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Optional, Union

from cjm_fasthtml_keyboard_navigation.core.navigation import NavigationPattern

## KeyboardMode

A named state that changes keyboard behavior. Modes can override navigation patterns, have their own key bindings, and trigger callbacks on entry/exit.

In [ ]:
#| export
@dataclass
class KeyboardMode:
    """A named mode that changes keyboard behavior."""

    # Identity
    name: str  # unique mode name (e.g., "navigation", "split", "audition")

    # Entry/exit keys
    enter_key: Optional[str] = None  # key to enter mode (None = programmatic only)
    enter_modifiers: frozenset[str] = field(
        default_factory=frozenset
    )  # modifiers required with enter_key
    exit_key: str = "Escape"  # key to exit mode
    exit_modifiers: frozenset[str] = field(
        default_factory=frozenset
    )  # modifiers required with exit_key

    # Conditions
    zone_ids: Optional[tuple[str, ...]] = None  # only available in these zones (None = all)

    # Behavior overrides
    navigation_override: Optional[NavigationPattern] = None  # override zone's navigation pattern

    # Callbacks (JS function names)
    on_enter: Optional[str] = None  # called when entering mode
    on_exit: Optional[str] = None  # called when exiting mode

    # Visual indicator
    indicator_text: Optional[str] = None  # text shown in UI when mode is active

    # Auto-exit conditions
    exit_on_zone_change: bool = True  # exit mode when switching zones

    def is_available_in_zone(
        self,
        zone_id: str  # the zone to check
    ) -> bool:        # True if mode is available in zone
        """Check if mode is available in given zone."""
        if self.zone_ids is None:
            return True
        return zone_id in self.zone_ids

    def to_js_config(self) -> dict: # JavaScript-compatible configuration
        """Convert to JavaScript configuration object."""
        return {
            "name": self.name,
            "enterKey": self.enter_key,
            "enterModifiers": list(self.enter_modifiers),
            "exitKey": self.exit_key,
            "exitModifiers": list(self.exit_modifiers),
            "zoneIds": list(self.zone_ids) if self.zone_ids else None,
            "navigationOverride": (
                self.navigation_override.name 
                if self.navigation_override else None
            ),
            "onEnter": self.on_enter,
            "onExit": self.on_exit,
            "indicatorText": self.indicator_text,
            "exitOnZoneChange": self.exit_on_zone_change,
        }

In [ ]:
# Test basic KeyboardMode
mode = KeyboardMode(
    name="split",
    enter_key="Enter",
    exit_key="Escape",
    zone_ids=("card-list",),
    on_enter="enterSplitMode",
    on_exit="exitSplitMode",
    indicator_text="Split Mode"
)

assert mode.name == "split"
assert mode.is_available_in_zone("card-list") == True
assert mode.is_available_in_zone("other-zone") == False

config = mode.to_js_config()
assert config["name"] == "split"
assert config["enterKey"] == "Enter"
assert config["zoneIds"] == ["card-list"]

In [ ]:
# Test mode available in all zones
global_mode = KeyboardMode(
    name="help",
    enter_key="?",
    zone_ids=None  # available everywhere
)

assert global_mode.is_available_in_zone("any-zone") == True
assert global_mode.to_js_config()["zoneIds"] is None

## Default Navigation Mode

The default mode that's always active when no other mode is entered.

In [ ]:
#| export
NAVIGATION_MODE = KeyboardMode(
    name="navigation",
    enter_key=None,  # default mode, cannot be entered via key
    exit_key="",     # cannot be exited (it's the default)
    indicator_text=None,
    exit_on_zone_change=False  # default mode persists across zone changes
)

In [ ]:
# Test default mode
assert NAVIGATION_MODE.name == "navigation"
assert NAVIGATION_MODE.enter_key is None
assert NAVIGATION_MODE.is_available_in_zone("any") == True

## Example Modes

Common mode patterns for reference.

In [ ]:
from cjm_fasthtml_keyboard_navigation.core.navigation import LinearHorizontal

# Split mode for text segmentation (Phase 2 style)
split_mode_example = KeyboardMode(
    name="split",
    enter_key="Enter",
    exit_key="Escape",
    navigation_override=LinearHorizontal(),  # left/right for caret
    on_enter="enterSplitMode",
    on_exit="exitSplitMode",
    indicator_text="Split Mode"
)

# Audition mode for audio preview (Phase 3 style)
audition_mode_example = KeyboardMode(
    name="audition",
    enter_key=None,  # entered by switching to VAD zone
    zone_ids=("vad-timeline",),
    on_enter="startAudition",
    indicator_text="Audition"
)

# Edit mode for inline editing
edit_mode_example = KeyboardMode(
    name="edit",
    enter_key="e",
    exit_key="Escape",
    on_enter="startEditing",
    on_exit="finishEditing",
    indicator_text="Edit"
)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()